## Download the Repository

[Repository Link](https://github.com/pijush2022/Lane_detection)

- This is our team's repository. This repository contains all the necessary code that we worked on and it also contains the dataset that we annotated.

- You do not need to do anything like uploading and adjusting the paths. Just run the cells sequentially.

- All the necessary commands are written in this notebook itself

In [1]:
!git clone https://github.com/pijush2022/Lane_detection

fatal: destination path 'road-detection' already exists and is not an empty directory.


## Install the Requirements

- Install all the python dependencies
- After Installing dependencies, Restart the runtime. If you do not restart the runtime, the python will throw "module not found error"

In [2]:
!pip install -r road-detection/TwinLiteNet/requirements.txt

## Copy Dataset from Repository

- Our repository contains dataset.zip in datasets folder in the repository. copy that zip file to root

In [3]:
!cp road-detection/datasets/dataset.zip ./

## Unzip the file

In [4]:
!unzip dataset.zip

Archive:  dataset.zip
replace dataset/test/images/road_image_160.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Import the all the required libraries

In [5]:
import torch
import cv2
import torch.utils.data
import torchvision.transforms as transforms
import numpy as np
import os
import random
import math
from matplotlib import pyplot as plt
import torch.nn as nn

## Image transformation functions

- By paper author

In [6]:
def augment_hsv(img, hgain=0.015, sgain=0.7, vgain=0.4):
    """change color hue, saturation, value"""
    r = np.random.uniform(-1, 1, 3) * [hgain, sgain, vgain] + 1  # random gains
    hue, sat, val = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2HSV))
    dtype = img.dtype  # uint8

    x = np.arange(0, 256, dtype=np.int16)
    lut_hue = ((x * r[0]) % 180).astype(dtype)
    lut_sat = np.clip(x * r[1], 0, 255).astype(dtype)
    lut_val = np.clip(x * r[2], 0, 255).astype(dtype)

    img_hsv = cv2.merge((cv2.LUT(hue, lut_hue), cv2.LUT(sat, lut_sat), cv2.LUT(val, lut_val))).astype(dtype)
    cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR, dst=img)  # no return needed

In [7]:
def random_perspective(combination,  degrees=10, translate=.1, scale=.1, shear=10, perspective=0.0, border=(0, 0)):
    """combination of img transform"""
    # torchvision.transforms.RandomAffine(degrees=(-10, 10), translate=(.1, .1), scale=(.9, 1.1), shear=(-10, 10))
    # targets = [cls, xyxy]
    img, gray, line = combination
    height = img.shape[0] + border[0] * 2  # shape(h,w,c)
    width = img.shape[1] + border[1] * 2

    # Center
    C = np.eye(3)
    C[0, 2] = -img.shape[1] / 2  # x translation (pixels)
    C[1, 2] = -img.shape[0] / 2  # y translation (pixels)

    # Perspective
    P = np.eye(3)
    P[2, 0] = random.uniform(-perspective, perspective)  # x perspective (about y)
    P[2, 1] = random.uniform(-perspective, perspective)  # y perspective (about x)

    # Rotation and Scale
    R = np.eye(3)
    a = random.uniform(-degrees, degrees)
    # a += random.choice([-180, -90, 0, 90])  # add 90deg rotations to small rotations
    s = random.uniform(1 - scale, 1 + scale)
    # s = 2 ** random.uniform(-scale, scale)
    R[:2] = cv2.getRotationMatrix2D(angle=a, center=(0, 0), scale=s)

    # Shear
    S = np.eye(3)
    S[0, 1] = math.tan(random.uniform(-shear, shear) * math.pi / 180)  # x shear (deg)
    S[1, 0] = math.tan(random.uniform(-shear, shear) * math.pi / 180)  # y shear (deg)

    # Translation
    T = np.eye(3)
    T[0, 2] = random.uniform(0.5 - translate, 0.5 + translate) * width  # x translation (pixels)
    T[1, 2] = random.uniform(0.5 - translate, 0.5 + translate) * height  # y translation (pixels)

    # Combined rotation matrix
    M = T @ S @ R @ P @ C  # order of operations (right to left) is IMPORTANT
    if (border[0] != 0) or (border[1] != 0) or (M != np.eye(3)).any():  # image changed
        if perspective:
            img = cv2.warpPerspective(img, M, dsize=(width, height), borderValue=(114, 114, 114))
            gray = cv2.warpPerspective(gray, M, dsize=(width, height), borderValue=0)
            line = cv2.warpPerspective(line, M, dsize=(width, height), borderValue=0)
        else:  # affine
            img = cv2.warpAffine(img, M[:2], dsize=(width, height), borderValue=(114, 114, 114))
            gray = cv2.warpAffine(gray, M[:2], dsize=(width, height), borderValue=0)
            line = cv2.warpAffine(line, M[:2], dsize=(width, height), borderValue=0)



    combination = (img, gray, line)
    return combination

In [8]:
!mkdir custom-dataset
!mkdir custom-dataset/test
!mkdir custom-dataset/train
!mkdir custom-dataset/validation

!mkdir -p custom-dataset/train/images
!mkdir -p custom-dataset/train/lane
!mkdir -p custom-dataset/train/segments
!mkdir -p custom-dataset/test/images
!mkdir -p custom-dataset/test/lane
!mkdir -p custom-dataset/test/segments
!mkdir -p custom-dataset/validation/images
!mkdir -p custom-dataset/validation/lane
!mkdir -p custom-dataset/validation/segments

# Copy only two images from each category in train
!cp dataset/train/images/road_image_0.png custom-dataset/train/images/
!cp dataset/train/lane/road_image_0.png custom-dataset/train/lane/
!cp dataset/train/segments/road_image_0.png custom-dataset/train/segments/

# Copy only two images from each category in train
!cp dataset/train/images/road_image_1.png custom-dataset/train/images/
!cp dataset/train/lane/road_image_1.png custom-dataset/train/lane/
!cp dataset/train/segments/road_image_1.png custom-dataset/train/segments/

# Copy only two images from each category in test
!cp dataset/test/images/* custom-dataset/test/images/
!cp dataset/test/lane/* custom-dataset/test/lane/
!cp dataset/test/segments/* custom-dataset/test/segments/


# Copy only two images from each category in validation
!cp dataset/validation/images/* custom-dataset/validation/images/
!cp dataset/validation/lane/* custom-dataset/validation/lane/
!cp dataset/validation/segments/* custom-dataset/validation/segments/

mkdir: cannot create directory ‘custom-dataset’: File exists
mkdir: cannot create directory ‘custom-dataset/test’: File exists
mkdir: cannot create directory ‘custom-dataset/train’: File exists
mkdir: cannot create directory ‘custom-dataset/validation’: File exists


## Custom Dataset Class

- This custom dataset class is based on the dataset class written by the author but with slight modifications like path. we have adjusted the path according to the google colab.

In [9]:
class MyDataset(torch.utils.data.Dataset):
    '''
    Class to load the dataset
    '''
    def __init__(self, transform=None, valid=False, test=False):
        '''
        :param imList: image list (Note that these lists have been processed and pickled using the loadData.py)
        :param labelList: label list (Note that these lists have been processed and pickled using the loadData.py)
        :param transform: Type of transformation. SEe Transforms.py for supported transformations
        '''

        self.transform = transform
        self.Tensor = transforms.ToTensor()
        self.valid=valid
        if valid:
            self.root='custom-dataset/validation/images'
            self.names=os.listdir(self.root)
        elif test:
            self.root='custom-dataset/test/images'
            self.names=os.listdir(self.root)
        else:
            self.root='custom-dataset/train/images/'
            self.names=os.listdir(self.root)

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        '''

        :param idx: Index of the image file
        :return: returns the image and corresponding label file.
        '''
        W_=640
        H_=360
        image_name=os.path.join(self.root,self.names[idx])

        image = cv2.imread(image_name)
        original_image = cv2.imread(image_name)
        label1 = cv2.imread(image_name.replace("images","segments").replace("jpg","png"), 0)
        label2 = cv2.imread(image_name.replace("images","lane").replace("jpg","png"), 0)
        if not self.valid:
            if random.random()<0.5:
                combination = (image, label1, label2)
                (image, label1, label2)= random_perspective(
                    combination=combination,
                    degrees=10,
                    translate=0.1,
                    scale=0.25,
                    shear=0.0
                )
            if random.random()<0.5:
                augment_hsv(image)
            if random.random() < 0.5:
                image = np.fliplr(image)
                label1 = np.fliplr(label1)
                label2 = np.fliplr(label2)

        label1 = cv2.resize(label1, (W_, H_))
        label2 = cv2.resize(label2, (W_, H_))
        image = cv2.resize(image, (W_, H_))

        _,seg_b1 = cv2.threshold(label1,1,255,cv2.THRESH_BINARY_INV)
        _,seg_b2 = cv2.threshold(label2,1,255,cv2.THRESH_BINARY_INV)
        _,seg1 = cv2.threshold(label1,1,255,cv2.THRESH_BINARY)
        _,seg2 = cv2.threshold(label2,1,255,cv2.THRESH_BINARY)

        seg1 = self.Tensor(seg1)
        seg2 = self.Tensor(seg2)
        seg_b1 = self.Tensor(seg_b1)
        seg_b2 = self.Tensor(seg_b2)
        seg_da = torch.stack((seg_b1[0], seg1[0]),0)
        seg_ll = torch.stack((seg_b2[0], seg2[0]),0)
        image = image[:, :, ::-1].transpose(2, 0, 1)
        image = np.ascontiguousarray(image)

        return original_image, image_name,torch.from_numpy(image),(seg_da,seg_ll)

## Intialize a dataloader

- Intialize a dataloader with batch size 8

- Intialize train, test, validation datasets.

In [10]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(MyDataset(), batch_size = 2, shuffle = True)
test_dataloader = DataLoader(MyDataset(test=True), batch_size = 8, shuffle = True)
val_dataloader = DataLoader(MyDataset(valid=True), batch_size = 8, shuffle = True)

## Take only two samples

In [11]:
_, _, input, target = next(iter(train_dataloader))

## Copy necessary files from repository

In [12]:
# Copy pretrained model from repository to root
!cp road-detection/TwinLiteNet/pretrained/best.pth ./

# Copy pytorch Neural Net from repo to root
!cp road-detection/TwinLiteNet/model/TwinLite.py ./

# Copy Loss function pytorch code from repo to root
!cp road-detection/TwinLiteNet/loss.py ./

# Copy all reqired constants from repo to root
!cp road-detection/TwinLiteNet/const.py ./

# Copy all val.py from repo to root
!cp road-detection/TwinLiteNet/val.py ./

## Mini Version of Original Network

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class MiniNet(nn.Module):
    def __init__(self):
        super(MiniNet, self).__init__()

        # Input Convolution
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()

        # Downsampling
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Intermediate Convolutions
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()

        # Upsampling
        self.upconv1 = nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.relu4 = nn.ReLU()

        self.upconv2 = nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(16)
        self.relu5 = nn.ReLU()

        # Classifiers
        self.classifier1 = nn.Conv2d(16, 2, kernel_size=1)
        self.classifier2 = nn.Conv2d(16, 2, kernel_size=1)

    def forward(self, x):
        # Input Convolution
        x = self.relu1(self.bn1(self.conv1(x)))

        # Downsampling
        x = self.pool(x)

        # Intermediate Convolutions
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu3(self.bn3(self.conv3(x)))

        # Upsampling
        x = self.relu4(self.bn4(self.upconv1(x)))
        x = self.relu5(self.bn5(self.upconv2(x)))

        # Classifiers
        classifier1 = self.classifier1(x)
        classifier2 = self.classifier2(x)

        return classifier1, classifier2

# Instantiate the model
model = MiniNet().to("cuda")


## Defining Optimizer and loss for training

In [14]:
from loss import TotalLoss

In [15]:
optimizer = torch.optim.Adam(model.parameters(), 5e-4, (0.9, 0.999), eps=1e-08, weight_decay=5e-4)
criterion = TotalLoss()

## Defining Custom metrics for evaluation

In [16]:
from tqdm import tqdm

class SegmentationMetric(object):
    '''
    imgLabel [batch_size, height(144), width(256)]
    confusionMatrix [[0(TN),1(FP)],
                     [2(FN),3(TP)]]
    '''
    def __init__(self, numClass):
        self.numClass = numClass
        self.confusionMatrix = np.zeros((self.numClass,)*2)

    def pixelAccuracy(self):
        # return all class overall pixel accuracy
        # acc = (TP + TN) / (TP + TN + FP + TN)
        acc = np.diag(self.confusionMatrix).sum() /  self.confusionMatrix.sum()
        return acc


    def classPixelAccuracy(self):
        # return each category pixel accuracy(A more accurate way to call it precision)
        # acc = (TP) / TP + FP
        classAcc = np.diag(self.confusionMatrix) / (self.confusionMatrix.sum(axis=0) + 1e-12)
        return classAcc

    def meanPixelAccuracy(self):
        classAcc = self.classPixelAccuracy()
        meanAcc = np.nanmean(classAcc)
        return meanAcc

    def meanIntersectionOverUnion(self):
        # Intersection = TP Union = TP + FP + FN
        # IoU = TP / (TP + FP + FN)
        intersection = np.diag(self.confusionMatrix)
        union = np.sum(self.confusionMatrix, axis=1) + np.sum(self.confusionMatrix, axis=0) - np.diag(self.confusionMatrix)
        IoU = intersection / union
        IoU[np.isnan(IoU)] = 0
        mIoU = np.nanmean(IoU)
        return mIoU

    def IntersectionOverUnion(self):
        intersection = np.diag(self.confusionMatrix)
        union = np.sum(self.confusionMatrix, axis=1) + np.sum(self.confusionMatrix, axis=0) - np.diag(self.confusionMatrix)
        IoU = intersection / union
        IoU[np.isnan(IoU)] = 0
        return IoU[1]

    def genConfusionMatrix(self, imgPredict, imgLabel):
        # remove classes from unlabeled pixels in gt image and predict
        # print(imgLabel.shape)
        mask = (imgLabel >= 0) & (imgLabel < self.numClass)
        label = self.numClass * imgLabel[mask] + imgPredict[mask]
        count = np.bincount(label, minlength=self.numClass**2)
        confusionMatrix = count.reshape(self.numClass, self.numClass)
        return confusionMatrix

    def Frequency_Weighted_Intersection_over_Union(self):
        # FWIOU =     [(TP+FN)/(TP+FP+TN+FN)] *[TP / (TP + FP + FN)]
        freq = np.sum(self.confusionMatrix, axis=1) / np.sum(self.confusionMatrix)
        iu = np.diag(self.confusionMatrix) / (
                np.sum(self.confusionMatrix, axis=1) + np.sum(self.confusionMatrix, axis=0) -
                np.diag(self.confusionMatrix))
        FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
        return FWIoU


    def addBatch(self, imgPredict, imgLabel):
        assert imgPredict.shape == imgLabel.shape
        self.confusionMatrix += self.genConfusionMatrix(imgPredict, imgLabel)

    def reset(self):
        self.confusionMatrix = np.zeros((self.numClass, self.numClass))

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count if self.count != 0 else 0

@torch.no_grad()
def val(val_loader, model):

    model.eval()


    DA=SegmentationMetric(2)
    LL=SegmentationMetric(2)

    da_acc_seg = AverageMeter()
    da_IoU_seg = AverageMeter()
    da_mIoU_seg = AverageMeter()

    ll_acc_seg = AverageMeter()
    ll_IoU_seg = AverageMeter()
    ll_mIoU_seg = AverageMeter()
    total_batches = len(val_loader)

    total_batches = len(val_loader)
    pbar = enumerate(val_loader)
    pbar = tqdm(pbar, total=total_batches)
    for i, (_, _,input, target) in pbar:
        input = input.cuda().float() / 255.0
            # target = target.cuda()

        input_var = input
        target_var = target

        # run the mdoel
        with torch.no_grad():
            output = model(input_var)

        out_da,out_ll=output
        target_da,target_ll=target

        _,da_predict=torch.max(out_da, 1)
        _,da_gt=torch.max(target_da, 1)

        _,ll_predict=torch.max(out_ll, 1)
        _,ll_gt=torch.max(target_ll, 1)
        DA.reset()
        DA.addBatch(da_predict.cpu(), da_gt.cpu())


        da_acc = DA.pixelAccuracy()
        da_IoU = DA.IntersectionOverUnion()
        da_mIoU = DA.meanIntersectionOverUnion()

        da_acc_seg.update(da_acc,input.size(0))
        da_IoU_seg.update(da_IoU,input.size(0))
        da_mIoU_seg.update(da_mIoU,input.size(0))


        LL.reset()
        LL.addBatch(ll_predict.cpu(), ll_gt.cpu())


        ll_acc = LL.pixelAccuracy()
        ll_IoU = LL.IntersectionOverUnion()
        ll_mIoU = LL.meanIntersectionOverUnion()

        ll_acc_seg.update(ll_acc,input.size(0))
        ll_IoU_seg.update(ll_IoU,input.size(0))
        ll_mIoU_seg.update(ll_mIoU,input.size(0))

    da_segment_result = (da_acc_seg.avg,da_IoU_seg.avg,da_mIoU_seg.avg)
    ll_segment_result = (ll_acc_seg.avg,ll_IoU_seg.avg,ll_mIoU_seg.avg)
    return da_segment_result,ll_segment_result

## Training The model

In [17]:
val_loss = []
train_loss = []

for i in list(range(1000)):
    model.train()
    model.to("cuda")
    output = model(input.cuda().float() / 255.0)

    # target=target.cuda()
    # print(target[0].size())
    optimizer.zero_grad()
    focal_loss, tversky_loss, loss = criterion(output,target)
    optimizer.zero_grad()
    train_loss.append(loss.item())
    # print(output.size())
    loss.backward()
    optimizer.step()
    if i % 50 == 0:
        print("loss: {loss:.5f}".format(loss = loss.item()))
        model.eval()
        example = torch.rand(1, 3, 360, 640).cuda()
        model = torch.jit.trace(model, example)
        print("Accuracy:")
        da_segment_results,ll_segment_results = val(train_dataloader, model)

        msg =  'Driving area Segment: Acc({da_seg_acc:.3f})\n' \
                            'Lane line Segment: Acc({ll_seg_acc:.3f})'.format(
                                da_seg_acc=da_segment_results[0],
                                ll_seg_acc=ll_segment_results[0])
        print(msg)
        print()
        print("Validation Evaluation:")
        da_segment_results,ll_segment_results = val(val_dataloader, model)

        msg =  'Driving area Segment: Acc({da_seg_acc:.3f})    IOU ({da_seg_iou:.3f})    mIOU({da_seg_miou:.3f})\n' \
                            'Lane line Segment: Acc({ll_seg_acc:.3f})    IOU ({ll_seg_iou:.3f})  mIOU({ll_seg_miou:.3f})'.format(
                                da_seg_acc=da_segment_results[0],da_seg_iou=da_segment_results[1],da_seg_miou=da_segment_results[2],
                                ll_seg_acc=ll_segment_results[0],ll_seg_iou=ll_segment_results[1],ll_seg_miou=ll_segment_results[2])
        print(msg)
        print("----------------------------------------------------------------")

loss: 1.21158
Accuracy:


100%|██████████| 1/1 [00:01<00:00,  1.36s/it]


Driving area Segment: Acc(0.856)
Lane line Segment: Acc(0.993)

Validation Evaluation:


100%|██████████| 3/3 [00:03<00:00,  1.12s/it]


Driving area Segment: Acc(0.798)    IOU (0.000)    mIOU(0.399)
Lane line Segment: Acc(0.981)    IOU (0.000)  mIOU(0.491)
----------------------------------------------------------------


...

Driving area Segment: Acc(0.848)    IOU (0.336)    mIOU(0.586)
Lane line Segment: Acc(0.961)    IOU (0.021)  mIOU(0.491)
----------------------------------------------------------------
loss: 0.02297
Accuracy:


100%|██████████| 1/1 [00:00<00:00,  8.45it/s]


Driving area Segment: Acc(0.988)
Lane line Segment: Acc(0.998)

Validation Evaluation:


100%|██████████| 3/3 [00:01<00:00,  2.61it/s]


Driving area Segment: Acc(0.851)    IOU (0.354)    mIOU(0.596)
Lane line Segment: Acc(0.958)    IOU (0.020)  mIOU(0.489)
----------------------------------------------------------------


## Evaluating the model on the test set

In [18]:
print("Test Evaluation:")
da_segment_results,ll_segment_results = val(test_dataloader, model)

msg =  '\nDriving area Segment: Acc({da_seg_acc:.3f})    IOU ({da_seg_iou:.3f})    mIOU({da_seg_miou:.3f})\n' \
                    'Lane line Segment: Acc({ll_seg_acc:.3f})    IOU ({ll_seg_iou:.3f})  mIOU({ll_seg_miou:.3f})'.format(
                        da_seg_acc=da_segment_results[0],da_seg_iou=da_segment_results[1],da_seg_miou=da_segment_results[2],
                        ll_seg_acc=ll_segment_results[0],ll_seg_iou=ll_segment_results[1],ll_seg_miou=ll_segment_results[2])
print(msg)
print("----------------------------------------------------------------")

Test Evaluation:


100%|██████████| 3/3 [00:01<00:00,  2.43it/s]


Driving area Segment: Acc(0.834)    IOU (0.226)    mIOU(0.526)
Lane line Segment: Acc(0.976)    IOU (0.024)  mIOU(0.500)
----------------------------------------------------------------
